https://www.kaggle.com/code/joonasyoon/wgan-cp-with-celeba-and-lsun-dataset/log

In [ ]:
from pathlib import Path
USE_COLAB: bool = True
dataset_base_path = Path("/content/drive/My Drive/ECE 792 - Advance Topics in Machine Learning/Datasets/FakeFaces/WGAN-CP")
if USE_COLAB:
  from google.colab import drive
  
  # Mount the drive to access google shared docs
  drive.mount('/content/drive/', force_remount=True)

  if dataset_base_path.exists():
    print("Folder exists")
  else:
    print("DOESN'T EXIST. Add desired folder as a shortcut in your 'My Drive'")

Mounted at /content/drive/
Folder exists


In [ ]:
dataset_base_path.mkdir(exist_ok=True, parents=True)
print(dataset_base_path.exists())

True


In [ ]:
from torch import nn
import torch
# Number of workers for dataloader
workers = 4

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 20

# Nubmer of loop to train ciritic
n_critics = 5

# Learning rate for optimizers
lr = 0.0001

# Weight Clipping Limit
weight_cliping_limit = 0.01

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

# Animation keyframes for result
fig_keyframes = 20

# Device
device = torch.device('cuda' if (torch.cuda.is_available() and ngpu > 0) else 'cpu')

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

In [ ]:
# custom weights initialization called on netG and netD
def weights_init(m):
  if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d)):
    nn.init.normal_(m.weight.data, 0.0, 0.02)
  elif isinstance(m, nn.BatchNorm2d):
    nn.init.normal_(m.weight.data, 1.0, 0.02)
    nn.init.constant_(m.bias.data, 0)

In [ ]:
# Generator Code
class Generator(nn.Module):
  def __init__(self):
    super(Generator, self).__init__()
    self.main = nn.Sequential(
      # input is Z, going into a convolution
      nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
      nn.BatchNorm2d(ngf * 8),
      nn.ReLU(True),
      # state size. (ngf*8) x 4 x 4
      nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
      nn.BatchNorm2d(ngf * 4),
      nn.ReLU(True),
      # state size. (ngf*4) x 8 x 8
      nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
      nn.BatchNorm2d(ngf * 2),
      nn.ReLU(True),
      # state size. (ngf*2) x 16 x 16
      nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
      nn.BatchNorm2d(ngf),
      nn.ReLU(True),
      # state size. (ngf) x 32 x 32
      nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
      nn.Tanh()
      # state size. (nc) x 64 x 64
    )

  def forward(self, input):
    return self.main(input)

In [ ]:
# Create the generator
netG = Generator().to(device)

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.02.
netG.apply(weights_init)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [ ]:
model_path = Path("/content/drive/My Drive/ECE 792 - Advance Topics in Machine Learning/Code/DatasetGeneration/WGGANCP/models/CelebA_netG.pth")
gen_model = torch.load(str(model_path))
netG.load_state_dict(gen_model)
netG.to(torch.device("cuda"))

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [ ]:
torch.manual_seed(999)
from torchvision.utils import save_image
from tqdm import tqdm
output_imgs_path = dataset_base_path

num_imgs_to_generate = 40000
batch_size = 1
iterations = int(num_imgs_to_generate / batch_size)
img_cnt = 0
for _ in tqdm(range(iterations)):
  with torch.no_grad():
    netG.eval()
    fakes = netG(torch.randn(batch_size, 100, 1, 1, dtype=torch.float, device=torch.device("cuda")))
    for fake in fakes:
      output_path = output_imgs_path / f"{img_cnt}.jpg"
      save_image(fake, output_path)
      img_cnt += 1

img_files = sorted(output_imgs_path.glob("*.jpg"))
print(f"Number of images generated: '{len(img_files)}'")
output_zip_path = output_imgs_path.parent / "WGAN-CP.zip"
from zipfile import ZipFile
from tqdm import tqdm

if output_zip_path.exists():
  print(f"Deleting '{output_zip_path}'")
  os.remove(str(output_zip_path))

print(f"Writing images to zip file '{output_zip_path}'")
with ZipFile(str(output_zip_path), mode='w') as archive:
  for file_ in tqdm(img_files):
    archive.write(file_)

with ZipFile(str(output_zip_path), mode='r') as zipObj:
  zipObj.extractall()

list_of_images_in_archive = zipObj.namelist()
print(f"Number of images in archive: '{len(list_of_images_in_archive)}'")
if len(list_of_images_in_archive) != len(img_files):
  raise RuntimeError(f"NOT ALL IMAGES WERE SUCCESSFULLY ARCHIVED")

drive.flush_and_unmount()

100%|██████████| 40000/40000 [03:54<00:00, 170.73it/s]


Number of images generated: '40000'
Writing images to zip file '/content/drive/My Drive/ECE 792 - Advance Topics in Machine Learning/Datasets/FakeFaces/DCGAN.zip'


100%|██████████| 40000/40000 [01:15<00:00, 528.93it/s]


Number of images in archive: '40000'


In [ ]:
output_imgs_path = dataset_base_path
import os

img_files = sorted(output_imgs_path.glob("*.jpg"))
print(f"Number of images generated: '{len(img_files)}'")
output_zip_path = output_imgs_path.parent / "WGAN-CP.zip"
from zipfile import ZipFile
from tqdm import tqdm

if output_zip_path.exists():
  print(f"Deleting '{output_zip_path}'")
  os.remove(str(output_zip_path))

print(f"Writing images to zip file '{output_zip_path}'")
with ZipFile(str(output_zip_path), mode='w') as archive:
  for file_ in tqdm(img_files):
    archive.write(file_)

with ZipFile(str(output_zip_path), mode='r') as zipObj:
  zipObj.extractall()

list_of_images_in_archive = zipObj.namelist()
print(f"Number of images in archive: '{len(list_of_images_in_archive)}'")
if len(list_of_images_in_archive) != len(img_files):
  raise RuntimeError(f"NOT ALL IMAGES WERE SUCCESSFULLY ARCHIVED")